A princípio, optei por utilizar um dicionário (hash map) global pela sua simplicidade e eficiência em armazenar diferentes datasets de maneira acessível para toda a aplicação, evitando a necessidade de reprocessamento contínuo. Além disso, essa abordagem me permite facilmente implementar uma rotina ou thread paralela para monitorar a fonte de dados CSV e detectar novas inserções. Quando houver atualizações, o CSV pode ser reprocessado, e o dicionário pode ser atualizado de forma eficiente e prática, mantendo o desempenho da aplicação.

In [ ]:
GLOBAL_DATASETS = {}

def get_global_dataset(dataset_name: str) -> pd.DataFrame:
    """
    Atualiza a variável global com um dataset tratado.

    :param dataset_name: Nome do dataset (chave) para atualização.
    :param df: DataFrame processado.
    """
    global GLOBAL_DATASETS
    return GLOBAL_DATASETS.get(dataset_name, None)


def add_global_dataset(dataset_name: str, df: pd.DataFrame) -> None:
    """
    Atualiza a variável global com um dataset tratado.

    :param dataset_name: Nome do dataset (chave) para atualização.
    :param df: DataFrame processado.
    """
    global GLOBAL_DATASETS
    GLOBAL_DATASETS[dataset_name] = df

if (job_postings_df := get_global_dataset(DatasetName.JOB_POSTINGS)) is None:
    job_postings_df = process_job_postings()

## Previsão de Vagas de Emprego para 2025
A função predict_job_postings_2025 é responsável por prever o número de postagens de vagas para o ano de 2025. Para isso, utiliza dados históricos de postagens de emprego, aplicando a biblioteca Prophet para modelar e prever os valores futuros. Prophet é uma ferramenta desenvolvida pelo Facebook para previsões de séries temporais, ideal para casos com sazonalidade, como o volume de postagens de empregos.

## Como a função funciona:
Filtragem por estado: Para cada estado, filtramos as postagens de emprego relacionadas ao estado específico.
Resample para frequência diária: A partir dos dados originais, as postagens são agrupadas em uma frequência diária usando a coluna listed_time_y_m_d, que indica a data da postagem.
Treinamento do modelo Prophet: Usamos o DataFrame diário para treinar o modelo Prophet e prever o número de vagas no futuro.
Previsão para 2025: Após treinar o modelo, geramos um DataFrame com datas futuras e realizamos a previsão para o ano de 2025.
Soma das previsões: A função soma o número total de vagas previstas para o ano de 2025.
Habilidades relacionadas: Para cada estado, também extraímos uma lista de habilidades relacionadas às vagas.

In [ ]:
def predict_job_postings_2025(predict_job_df: pd.DataFrame) -> pd.DataFrame:
    states = predict_job_df['state'].unique()
    predict_job_df['ds'] = predict_job_df['listed_time_y_m_d']

    results = []

    for state in states:
        df_state: pd.DataFrame = predict_job_df[predict_job_df['state'] ==
                                                state]

        df_weekly = df_state.set_index('ds').resample('D').size().reset_index(
            name='y')

        if df_weekly.empty or df_weekly.shape[0] < 2:
            skills_list = ', '.join(df_state['skill_abr'].dropna().unique())
            results.append({
                'state': state,
                'predicted_postings': 0,
                'skill_abr': skills_list
            })
            continue

        model = Prophet()
        model.fit(df_weekly)

        future = model.make_future_dataframe(periods=365, freq='D')

        forecast = model.predict(future)

        total_jobs_2025 = forecast[(forecast['ds'] >= '2025-01-01') & (
            forecast['ds'] <= '2025-12-31')]['yhat_lower'].sum()

        total_jobs_2025 = round(total_jobs_2025, 0)

        skills_list = ', '.join(df_state['skill_abr'].dropna().unique())

        results.append({
            'state': state,
            'predicted_postings': total_jobs_2025,
            'skill_abr': skills_list
        })

    for state in ALL_STATES:
        if state not in [result['state'] for result in results]:
            results.append({
                'state': state,
                'predicted_postings': 0,
                'skill_abr': ''
            })

    return pd.DataFrame(results)